<a href="https://colab.research.google.com/github/JJJJeon/ESAA/blob/main/OB_9%EC%A3%BC%EC%B0%A8_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **문서 군집화 소개와 실습(Opinion Review 데이터 세트)**
**문서 군집화(Document Clustering)**: 비슷한 텍스트 구성의 문서를 군집화(Clustering)



*   동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류
*   학습 데이터 세트가 필요없는 비지도학습 기반으로 동작



### Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기
전체 51개의 파일이 토요타와 같은 자동차 브랜드에 대한 평가와 아이팟 나노의 음질과 같은 다양한 전자 제품과 호텔 서비스 등에 대한 리뷰 내용

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
import pandas as pd
import glob, os

# 압축 파일을 풀어놓은 디렉터리
path=r'/content/drive/MyDrive/readcsv/topics'
# path로 지정한 디렉터리 밑에 있는 모든 .data 파일의 파일명을 리스트로 취합
all_files=glob.glob(os.path.join(path, '*.data'))
filename_list=[]
opinion_text=[]

# 개별 파일의 파일명은 filename_list로 취합,
# 개별 파일의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환해 opinion_text list로 취합
for file_ in all_files:
  # 개별 파일을 읽어서 DataFrame으로 생성
  df=pd.read_table(file_, index_col=None, header=0, encoding='latin1')

  # 절대 경로로 주어진 파일명을 가공
  # 맨 마지막 .data 확장자도 제거
  filename_=file_.split('/')[-1]
  filename=filename_.split('.')[0]

  # 파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가
  filename_list.append(filename)
  opinion_text.append(df.to_string())

# 파일명 list와 파일 내용 list 객체를 DataFrame으로 생성
document_df=pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,comfort_honda_accord_2008,...
1,buttons_amazon_kindle,...
2,battery-life_netbook_1005ha,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_amazon_kindle,...


In [8]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# 단어 원형 추출 함수
lemmar = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# 특수 문자 사전 생성: {33: None ...}
# ord(): 아스키 코드 생성
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# 특수 문자 제거 및 단어 원형 추출
def LemNormalize(text):
    # 텍스트 소문자 변경 후 특수 문자 제거
    text_new = text.lower().translate(remove_punct_dict)
    
    # 단어 토큰화
    word_tokens = nltk.word_tokenize(text_new)
    
    # 단어 원형 추출
    return LemTokens(word_tokens)

In [9]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [11]:
# TF-IDF 형태로 피처 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect=TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                           ngram_range=(1,2), min_df=0.05, max_df=0.85)
# opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect=tfidf_vect.fit_transform(document_df['opinion_text']) # 개별 문서 텍스트에 대해 TF-IDF 변환된 피처 벡터화된 행렬 구할 수 있음

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [13]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행
km_cluster=KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label=km_cluster.labels_
cluster_centers=km_cluster.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [14]:
document_df['cluster_label']=cluster_label
document_df.head()
# 각 데이터별로 할당된 군집의 레이블

,filename,opinion_text,cluster_label
0,comfort_honda_accord_2008,...,4
1,buttons_amazon_kindle,...,3
2,battery-life_netbook_1005ha,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_amazon_kindle,...,1


In [16]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')
# Cluster #0: 호텔에 대한 리뷰

,filename,opinion_text,cluster_label
5,bathroom_bestwestern_hotel_sfo,...,0
7,food_holiday_inn_london,...,0
14,food_swissotel_chicago,...,0
30,free_bestwestern_hotel_sfo,...,0
19,location_bestwestern_hotel_sfo,...,0
17,location_holiday_inn_london,...,0
26,parking_bestwestern_hotel_sfo,...,0
22,price_holiday_inn_london,...,0
21,room_holiday_inn_london,...,0
41,rooms_bestwestern_hotel_sfo,...,0


In [17]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')
# Cluster #1: 킨들, 아이팟, 넷북 등의 포터블 전자기기에 대한 리뷰

,filename,opinion_text,cluster_label
4,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
2,battery-life_netbook_1005ha,...,1
8,features_windows7,...,1
27,performance_honda_accord_2008,...,1
18,performance_netbook_1005ha,...,1
43,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,1
33,speed_windows7,...,1
49,video_ipod_nano_8gb,...,1


In [18]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')
# Cluster #2: Cluster #1과 비슷하게 아이팟, 넷북이 군집에 포함되어 있지만, 주로 차량용 네비게이션

,filename,opinion_text,cluster_label
6,accuracy_garmin_nuvi_255W_gps,...,2
13,directions_garmin_nuvi_255W_gps,...,2
11,display_garmin_nuvi_255W_gps,...,2
29,keyboard_netbook_1005ha,...,2
38,satellite_garmin_nuvi_255W_gps,...,2
36,screen_garmin_nuvi_255W_gps,...,2
34,screen_ipod_nano_8gb,...,2
37,screen_netbook_1005ha,...,2
42,size_asus_netbook_1005ha,...,2
39,speed_garmin_nuvi_255W_gps,...,2


In [21]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')
# Cluster #3: 킨들 리뷰

,filename,opinion_text,cluster_label
1,buttons_amazon_kindle,...,3
10,eyesight-issues_amazon_kindle,...,3
9,fonts_amazon_kindle,...,3
16,navigation_amazon_kindle,...,3
15,price_amazon_kindle,...,3


In [20]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')
# Cluster #4: 토요타와 혼다 등의 자동차에 대한 리뷰

,filename,opinion_text,cluster_label
0,comfort_honda_accord_2008,...,4
12,comfort_toyota_camry_2007,...,4
25,gas_mileage_toyota_camry_2007,...,4
20,interior_honda_accord_2008,...,4
28,interior_toyota_camry_2007,...,4
24,mileage_honda_accord_2008,...,4
23,quality_toyota_camry_2007,...,4
40,seats_honda_accord_2008,...,4
45,transmission_toyota_camry_2007,...,4


군집 개수가 약간 많게 설정돼 있어서 세분화되어 군집화된 경향 있음 -> 중심 개수를 5개에서 3개로 낮춰서 3개 그룹으로 군집화

In [22]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화
km_cluster=KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label=km_cluster.labels_
cluster_centers=km_cluster.cluster_centers_

# 소속 군집을 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label']=cluster_label
document_df.sort_values(by='cluster_label')

# Cluster #0: 자동차 리뷰, #1: 호텔 리뷰, #2: 포터블 전자기기 리뷰

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
0,comfort_honda_accord_2008,...,0
45,transmission_toyota_camry_2007,...,0
40,seats_honda_accord_2008,...,0
28,interior_toyota_camry_2007,...,0
27,performance_honda_accord_2008,...,0
24,mileage_honda_accord_2008,...,0
23,quality_toyota_camry_2007,...,0
20,interior_honda_accord_2008,...,0
12,comfort_toyota_camry_2007,...,0
25,gas_mileage_toyota_camry_2007,...,0


### 군집별 핵심 단어 추출하기
KMeans의 clusters_centers_: 각 군집을 구성하는 단어 피처가 군집의 중심을 기준으로 얼마나 가깝게 위치해 있는지



*   행: 개별 군집
*   열: 개별 피처

개별 군집 내의 상대 위치를 숫자 값으로 표현한 일종의 좌표 값. 1에 가까울수록 중심과 가까운 값 의미

In [23]:
cluster_centers=km_cluster.cluster_centers_
print('cluster_centers shape:', cluster_centers.shape)
print(cluster_centers)
# 군집이 3개, word 피처가 4611개

cluster_centers shape: (3, 4611)
[[0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]]


In [24]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함
def get_cluster_details(cluster_model, cluster_data, feature_names, cluster_num, top_n_features=10):
  cluster_details={}

  # cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값을 반환
  # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
  centroid_feature_ordered_ind=cluster_model.cluster_centers_.argsort()[:,::-1]

  # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(cluster_num):
    # 개별 군집별 정보를 담을 데이터 초기화
    cluster_details[cluster_num]={}
    cluster_details[cluster_num]['cluster']=cluster_num

    # cluster_centers_.argsort()[:,::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함
    top_feature_indexes=centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features=[feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
    top_feature_values=cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features']=top_features
    cluster_details[cluster_num]['top_features_value']=top_feature_values
    filenames=cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
    filenames=filenames.values.tolist()

    cluster_details[cluster_num]['filenames']=filenames

  return cluster_details

In [33]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('###### Cluster {0}'.format(cluster_num))
    print('Top features:',cluster_detail['top_features'])
    print('Reviews 파일명:',cluster_detail['filenames'][:7])
    print('=============================================')

In [34]:
feature_names=tfidf_vect.get_feature_names_out()

cluster_details=get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                    feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

###### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명: ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'interior_honda_accord_2008', 'quality_toyota_camry_2007', 'mileage_honda_accord_2008', 'gas_mileage_toyota_camry_2007', 'performance_honda_accord_2008']
###### Cluster 1
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명: ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'location_holiday_inn_london', 'location_bestwestern_hotel_sfo', 'room_holiday_inn_london', 'price_holiday_inn_london']
###### Cluster 2
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명: ['buttons_amazon_kindle', 'battery-life_netbook_1005ha', 'battery-life_ipod_nano_8gb', 'battery-life_amazon_kindle', 'accur

In [29]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

In [30]:
feature_names=tfidf_vect.get_feature_names_out()

cluster_details=get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                    feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

###### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']


KeyError: ignored